In [19]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['화자 인식', '대화 주제 식별', 'LM(한국어)', 'dialog']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
pattern = r'\((.*?)\)\/\((.*?)\)'
replacement = r'\2'

pattern2 = r'[a-z0-9]/|\(|\)'
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)['dataSet']
            
            #### data preprocess
            preprocessed_dialog = []
            for utt in line['dialogs']:
                new_utt = {}
                text = utt['text']
                result = re.sub(pattern, replacement, text)
                result = re.sub(pattern2, '', result).replace('  ',' ').strip()
                new_utt['speaker'] = utt['speaker']
                new_utt['text'] = result
                preprocessed_dialog.append(new_utt)
                
            #### data preprocess end 

            #### 화자 인식
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = {}
            data['output'] = {}
            data['input']['typeInfo'] = line['typeInfo']
            input_dialogs = []
            output_speaker_id = []
            for utt in preprocessed_dialog:
                if utt['text']:
                    input_dialogs.append(utt['text'])
                    output_speaker_id.append(utt['speaker'])
            data['input']['dialogs'] = input_dialogs
            data['output']['speaker'] = output_speaker_id
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 화자 인식 end


            #### 대화 주제 식별
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = {}
            data['output'] = {}
            data['input']['typeInfo'] = line['typeInfo'].copy()
            category = data['input']['typeInfo'].pop('category')
            subcategory = data['input']['typeInfo'].pop('subcategory')
            topic = category + ' ' + subcategory
            data['input']['dialog'] = preprocessed_dialog
            data['output']['topic'] = topic
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 대화 주제 식별 end


            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            data['text'] = ' '.join([utt['text'] for utt in preprocessed_dialog])
            data['text'] = data['text'].replace('  ', ' ').strip()
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end


            #### dialog
            data = {'text': None}
            ## preprocess data from line
            data['text'] = {}
            data['text']['typeInfo'] = line['typeInfo']
            data['text']['dialog'] = preprocessed_dialog
            ## preprocess data from line end
            task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### dialog end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 4/4 [00:00<00:00, 13.56it/s]
